<a href="https://colab.research.google.com/github/dilaragokay/3D-point-capsule-networks/blob/colab/Perturbation_3DCapsNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install dependencies

In [ ]:
!sudo apt-get install libhdf5-dev
!sudo pip install h5py
!pip install open3d
!pip install gdown

In [ ]:
%cd models/nndistance
!python build.py install

# 2. Install ShapeNetPart Dataset

In [ ]:
%cd ../../dataset
!bash download_shapenet_part16_catagories.sh

# 3. Download checkpoints

In [ ]:
!mkdir ../checkpoints
%cd ../checkpoints
# Checkpoint link is obtained from https://github.com/yongheng1991/3D-point-capsule-networks/blob/master/README.md
!gdown https://drive.google.com/uc?id=1so0OiVPS93n7Ed36yiMHQb-u0f1zJSzJ

# 4. Perturb example point cloud
<img src="https://i.ibb.co/jwGwVSV/Screenshot-2020-11-05-at-00-58-39.png">

*Image source: [1]*

* I picked one object (plane) for the sake of simplicity.

* After dynamic routing, 64 latent capsules are obtained. Since some of the capsules seem to have irregular points, plus, visualizing all of the capsules is not feasible, I've picked 3 capsules which seem to represent the object well. These capsules represent **wings, rudder, and bottom**. 
* Each capsule is a 64x1 vector. I've picked 3 features from these capsules. I've picked the ones on position 10, 20, and 30 (with no particular reason). [2] perturbs features in the capsules within [-0.5, 0.5]. I chose {-0.25, 0.25} in order not to have too many combinations to visualize.
* At each attempt, I've changed one of the variables above and kept the rest constant. In a nutshell,

```
for capsule in capsule_set:
    for feature in feature_set:
        for amount in perturbations:
            perturb the feature in the capsule by amount then save
```

Changes are in `viz_perturbation.py`.



In [ ]:
%cd ../../mini_example/AE/

In [ ]:
!python viz_perturbation.py --model ../../checkpoints/shapenet_part_dataset_ae_200.pth

Namespace(batch_size=8, latent_caps_size=64, latent_vec_size=64, model='../../checkpoints/shapenet_part_dataset_ae_200.pth', num_points=2048, prim_caps_size=1024, prim_vec_size=16)


## 5. Show results in 3D plot

In [ ]:
import open3d as o3d
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import glob

In [ ]:
pcd_files = glob.glob('*.pcd')

In [ ]:
dictionary = {}
for f in pcd_files:
  key = '_'.join(f.split('_')[:2])
  if key in dictionary:
    dictionary[key].append(f)
  else:
    dictionary[key] = [f]

for key in dictionary:
  dictionary[key] = sorted(dictionary[key])

### Visualize the first point cloud

In [ ]:
def plot_perturbations(minus_pert, zero_pert, plus_pert, specs):
  # Initialize figure with 9 3D subplots
  fig = make_subplots(
      rows=3,
      cols=1,
      specs=[[{"type": "scene"}], [{"type": "scene"}], [{"type": "scene"}]],
      shared_xaxes=True,
      subplot_titles=("-0.25 perturbation", "No perturbation", "+0.25 perturbation"))

  # Generate data
  pcd = o3d.io.read_point_cloud(minus_pert)
  x1 = np.asarray(pcd.points)[:, 0]
  y1 = np.asarray(pcd.points)[:, 1]
  z1 = np.asarray(pcd.points)[:, 2]

  c1 = np.asarray(pcd.colors)

  pcd = o3d.io.read_point_cloud(zero_pert)
  x2 = np.asarray(pcd.points)[:, 0]
  y2 = np.asarray(pcd.points)[:, 1]
  z2 = np.asarray(pcd.points)[:, 2]

  c2 = np.asarray(pcd.colors)

  pcd = o3d.io.read_point_cloud(plus_pert)
  x3 = np.asarray(pcd.points)[:, 0]
  y3 = np.asarray(pcd.points)[:, 1]
  z3 = np.asarray(pcd.points)[:, 2]

  c3 = np.asarray(pcd.colors)

  # adding point clouds to subplots.
  fig.add_trace(go.Scatter3d(x=x1, y=y1, z=z1,mode='markers',
                                    marker=dict(
          size=2,
          color=c1,                # set color to an array/list of desired values
          opacity=0.4
      )),
      row=1, col=1)

  fig.add_trace(go.Scatter3d(x=x2, y=y2, z=z2, mode='markers',
                                    marker=dict(
          size=2,
          color=c2,                # set color to an array/list of desired values
          opacity=0.4
      )),
      row=2, col=1)

  fig.add_trace(go.Scatter3d(x=x3, y=y3, z=z3, mode='markers',
                                    marker=dict(
          size=2,
          color=c3,                # set color to an array/list of desired values
          opacity=0.4
      )),
      row=3, col=1)

  fig.update_layout(
      title_text=specs,
      height=1000,
      width=1000
  )

  fig.show()

In [ ]:
key = list(dictionary.keys())[0]  # key for the first point cloud

point_clouds = dictionary[key]

num_to_color = {
    '29': 'red',
    '40': 'green',
    '16': 'blue'
}

num_to_name = {
    '29': 'rudder',
    '40': 'wings',
    '16': 'bottom'
}
i = 0

Run the following cell as long as there are more combinations. All point clouds are not visualized at once as it can crash the browser.

In [ ]:
if i < len(point_clouds):
  minus = point_clouds[i]
  plus = point_clouds[i+1]
  zero = point_clouds[i+2]
  name_splitted = plus.split('_')
  caps_no = name_splitted[2][4:]
  feat_no = name_splitted[3][4:]
  amount = name_splitted[5][:-4]
  specs = "Feature #{} of the capsule for {}. Capsule is shown in {}".format(
      feat_no,
      num_to_name[caps_no],
      num_to_color[caps_no])
  plot_perturbations(minus, zero, plus, specs)
  i+=3

# 6. Visualize precaptured images from 3 different viewpoints

In images below, each 3x3 figure shows how changing certain feature in a certain capsule affects view from side, front, and top of the plane. In short, each subfigure has been organized the following way.  

|       | side view | front view | top view |
|-------|-----------|------------|----------|
| -0.25 |      img     |     img       |    img      |
| 0     |      img     |          img  |       img   |
| 0.25  |      img     |         img   |       img   |

Note that these figures have been captured by hand so not all these images have been captured from the same angle i.e. some images might be slightly rotated.

## 6.1. Change in capsule that corresponds to the bottom of the plane (blue points)
### 6.1.1. Perturbation on feature #10

<img src="https://drive.google.com/uc?export=view&id=1ZJpBLMVfL3R1pLoge1ZiDrXN2OJiF_OX">

### 6.1.2. Perturbation on feature #20

<img src="https://drive.google.com/uc?export=view&id=1tVz8mhOhHFsJbvZR3XnUzLSHxvHvzXws">

### 6.1.3. Perturbation on feature #30

<img src="https://drive.google.com/uc?export=view&id=1fhTCQXEP25SrFT1kMR0ZrKoXFenRkYjC">

## 6.2. Change in capsule that corresponds to the rudder of the plane (red points)
### 6.2.1. Perturbation on feature #10

<img src="https://drive.google.com/uc?export=view&id=1uELPomS0IZw9IqrRoF5uCui6_uQdd8nr">

### 6.2.2. Perturbation on feature #20

<img src="https://drive.google.com/uc?export=view&id=1yaLi7Ed7mI9ttixmxcAMcdlWub421vrd">

### 6.2.3. Perturbation on feature #30

<img src="https://drive.google.com/uc?export=view&id=1jsvh0CbT263KbtarXo8zFAdxwdpwvIpr">

## 6.3. Change in capsule that corresponds to the wings of the plane (green points)
### 6.3.1. Perturbation on feature #10

<img src="https://drive.google.com/uc?export=view&id=15VStNiDnyovnnWYCX9xQfs9hkdPHUul2">

### 6.3.2. Perturbation on feature #20

<img src="https://drive.google.com/uc?export=view&id=1Z_kIv7yCX5inAnlQeQ5gxD06CATFjgkd">

### 6.3.3. Perturbation on feature #30

<img src="https://drive.google.com/uc?export=view&id=1C87QsMgY0I8lsFIvaaU2PyJSm-vTU9kT">

# 7. Observations (w/ [@evinpinar](https://github.com/evinpinar))

* Changes in feature #30 of rudder,
* Changes in feature #20 of wing,   
* Changes in feature #30 of wing   
causes rotation and translation.

# Reference

[1] Zhao, Yongheng, et al. "3D point capsule networks." *Proceedings of the IEEE conference on computer vision and pattern recognition*. 2019.  
[2] Sabour, Sara, Nicholas Frosst, and Geoffrey E. Hinton. "Dynamic routing between capsules." *Advances in neural information processing systems*. 2017.